In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [143]:
salaries = pd.read_excel('salaries_1985to2020_final.xlsx')
p_stats = pd.read_excel('Complete_Train_Set_With_Cap_Usage.xlsx')
stats_2020 = pd.read_excel('Clean_2019_2020_Data.xlsx')

In [144]:
salary_file = pd.ExcelFile('salaries_1985to2020_final.xlsx')
players_info = pd.read_excel(salary_file, 1)

In [145]:
salary_caps = pd.read_excel('salary_caps.xlsx')

In [146]:
salaries = salaries.rename(columns={'names':'name'})
stats_2020 = stats_2020.merge(salaries.loc[salaries['season_end']==2020], on=['name'])
salary_cap_2020 = salary_caps[salary_caps['season']=='2019-20']['salary_cap']
individual_salaries = stats_2020['salary']
cap_usage = individual_salaries/int(salary_cap_2020)
stats_2020['cap_usage'] = cap_usage

In [147]:
years = list(salaries.groupby('player_id').apply(lambda x: [i for i in range(1, len(x)+1)]))
flatten = lambda l: [item for sublist in l for item in sublist]
salaries['years'] = flatten(years)

In [148]:
salaries.head()

,name,player_id,salary,season,season_end,season_start,team,key,years
0,Alaa Abdelnaby,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers,Alaa Abdelnaby1990-91,1
1,Alaa Abdelnaby,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers,Alaa Abdelnaby1991-92,2
2,Alaa Abdelnaby,abdelal01,500000,1992-93,1993,1992,Boston Celtics,Alaa Abdelnaby1992-93,3
3,Alaa Abdelnaby,abdelal01,805000,1993-94,1994,1993,Boston Celtics,Alaa Abdelnaby1993-94,4
4,Alaa Abdelnaby,abdelal01,650000,1994-95,1995,1994,Sacramento Kings,Alaa Abdelnaby1994-95,5


In [149]:
salaries.loc[salaries['name']=='LeBron James']

,name,player_id,salary,season,season_end,season_start,team,key,years
6787,LeBron James,jamesle01,4018920,2003-04,2004,2003,Cleveland Cavaliers,LeBron James2003-04,1
6788,LeBron James,jamesle01,4320360,2004-05,2005,2004,Cleveland Cavaliers,LeBron James2004-05,2
6789,LeBron James,jamesle01,4621800,2005-06,2006,2005,Cleveland Cavaliers,LeBron James2005-06,3
6790,LeBron James,jamesle01,5828090,2006-07,2007,2006,Cleveland Cavaliers,LeBron James2006-07,4
6791,LeBron James,jamesle01,13041250,2007-08,2008,2007,Cleveland Cavaliers,LeBron James2007-08,5
6792,LeBron James,jamesle01,14410581,2008-09,2009,2008,Cleveland Cavaliers,LeBron James2008-09,6
6793,LeBron James,jamesle01,15779912,2009-10,2010,2009,Cleveland Cavaliers,LeBron James2009-10,7
6794,LeBron James,jamesle01,14500000,2010-11,2011,2010,Miami Heat,LeBron James2010-11,8
6795,LeBron James,jamesle01,16022500,2011-12,2012,2011,Miami Heat,LeBron James2011-12,9
6796,LeBron James,jamesle01,17545000,2012-13,2013,2012,Miami Heat,LeBron James2012-13,10


In [150]:
p_stats.head()

,Unnamed: 0,name,positions,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,season,player_id,salary,season_end,season_start,team_y,key,salary_cap_x,salary_cap_y,cap_usage
0,0,Kareem Abdul-Jabbar,CENTER,37,LOS ANGELES LAKERS,79,1.000000,33.291139,9.151899,15.278481,...,1984-85,abdulka01,1530000,1985,1984,Los Angeles Lakers,Kareem Abdul-Jabbar1984-85,3600000,3600000,0.425000
1,1,Alvan Adams,POWER FORWARD,30,PHOENIX SUNS,82,0.841463,26.048780,5.804878,11.158537,...,1984-85,adamsal01,500000,1985,1984,Phoenix Suns,Alvan Adams1984-85,3600000,3600000,0.138889
2,2,Mark Aguirre,SMALL FORWARD,25,DALLAS MAVERICKS,80,0.987500,33.737500,9.925000,19.612500,...,1984-85,aguirma01,800000,1985,1984,Dallas Mavericks,Mark Aguirre1984-85,3600000,3600000,0.222222
3,3,Danny Ainge,SHOOTING GUARD,25,BOSTON CELTICS,75,0.973333,34.186667,5.586667,10.560000,...,1984-85,aingeda01,400000,1985,1984,Boston Celtics,Danny Ainge1984-85,3600000,3600000,0.111111
4,4,John Bagley,POINT GUARD,24,CLEVELAND CAVALIERS,81,0.802469,29.641975,4.172840,8.555556,...,1984-85,baglejo01,209000,1985,1984,Cleveland Cavaliers,John Bagley1984-85,3600000,3600000,0.058056


In [151]:
p_stats = p_stats.merge(salaries, on = ['name', 'season'])
p_stats.head()

,Unnamed: 0,name,positions,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,salary_cap_x,salary_cap_y,cap_usage,player_id_y,salary_y,season_end_y,season_start_y,team,key_y,years
0,0,Kareem Abdul-Jabbar,CENTER,37,LOS ANGELES LAKERS,79,1.000000,33.291139,9.151899,15.278481,...,3600000,3600000,0.425000,abdulka01,1530000,1985,1984,Los Angeles Lakers,Kareem Abdul-Jabbar1984-85,1
1,1,Alvan Adams,POWER FORWARD,30,PHOENIX SUNS,82,0.841463,26.048780,5.804878,11.158537,...,3600000,3600000,0.138889,adamsal01,500000,1985,1984,Phoenix Suns,Alvan Adams1984-85,1
2,2,Mark Aguirre,SMALL FORWARD,25,DALLAS MAVERICKS,80,0.987500,33.737500,9.925000,19.612500,...,3600000,3600000,0.222222,aguirma01,800000,1985,1984,Dallas Mavericks,Mark Aguirre1984-85,1
3,3,Danny Ainge,SHOOTING GUARD,25,BOSTON CELTICS,75,0.973333,34.186667,5.586667,10.560000,...,3600000,3600000,0.111111,aingeda01,400000,1985,1984,Boston Celtics,Danny Ainge1984-85,1
4,4,John Bagley,POINT GUARD,24,CLEVELAND CAVALIERS,81,0.802469,29.641975,4.172840,8.555556,...,3600000,3600000,0.058056,baglejo01,209000,1985,1984,Cleveland Cavaliers,John Bagley1984-85,1


In [152]:
stats_2020 = stats_2020.merge(salaries, on = ['name', 'season'])
stats_2020.head()

,Unnamed: 0,name,positions,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,team_y,key_x,cap_usage,player_id_y,salary_y,season_end_y,season_start_y,team,key_y,years
0,0,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,...,Oklahoma City Thunder,Steven Adams2019-20,0.236785,adamsst01,25842697,2020,2019,Oklahoma City Thunder,Steven Adams2019-20,7
1,1,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,...,Miami Heat,Bam Adebayo2019-20,0.031648,adebaba01,3454080,2020,2019,Miami Heat,Bam Adebayo2019-20,3
2,2,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,...,San Antonio Kings,LaMarcus Aldridge2019-20,0.238226,aldrila01,26000000,2020,2019,San Antonio Kings,LaMarcus Aldridge2019-20,14
3,4,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,...,New Orleans Pelicans,Nickeil Alexander-Walker2019-20,0.027165,alexani01,2964840,2020,2019,New Orleans Pelicans,Nickeil Alexander-Walker2019-20,1
4,5,Grayson Allen,SG,24,MEM,38,0,18.9,3.1,6.6,...,Memphis Grizzlies,Grayson Allen2019-20,0.022259,allengr01,2429400,2020,2019,Memphis Grizzlies,Grayson Allen2019-20,2


In [156]:
p_stats.columns

Index(['Unnamed: 0', 'name', 'positions', 'age', 'team_x', 'games_played',
       'games_started', 'minutes_played', 'made_field_goals',
       'attempted_field_goals', 'made_three_point_field_goals',
       'attempted_three_point_field_goals', 'made_free_throws',
       'attempted_free_throws', 'offensive_rebounds', 'defensive_rebounds',
       'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points',
       'player_efficiency_rating', 'true_shooting_percentage',
       'three_point_attempt_rate', 'free_throw_attempt_rate',
       'offensive_rebound_percentage', 'defensive_rebound_percentage',
       'total_rebound_percentage', 'assist_percentage', 'steal_percentage',
       'block_percentage', 'turnover_percentage', 'usage_percentage',
       'offensive_win_shares', 'defensive_win_shares', 'win_shares',
       'win_shares_per_48_minutes', 'offensive_box_plus_minus',
       'defensive_box_plus_minus', 'box_plus_minus',
       'value_over_replacement_player', 'season', 'p

In [153]:
p_stats_v2 = p_stats.drop(columns=['team_y', 'key_x', 'player_id_y', 'salary_y', 'season_end_y',
       'season_start_y', 'team', 'key_y'])
stats_2020_v2 = stats_2020.drop(columns=['team_y', 'key_x', 'player_id_y', 'salary_y', 'season_end_y',
       'season_start_y', 'team', 'key_y'])

# model starts from here

In [172]:
def identify_features_linear(X, y):
    rankings_df = pd.DataFrame(columns=['feature', 'r_squared_value'])
    for feature in X.columns.values:
        model = LinearRegression().fit(X[[feature]], y)
        rankings_df = rankings_df.append({'feature': feature, 'r_squared_value': model.score(X[[feature]], y)}, ignore_index=True)
    return rankings_df.sort_values(by=['r_squared_value'])
identify_features_linear(extract_numerical_data(p_stats), y_train)

,feature,r_squared_value
29,turnover_percentage,1.724975e-07
1,age,3.451533e-04
43,salary_cap_y,8.699866e-04
42,salary_cap_x,8.699866e-04
0,Unnamed: 0,1.134273e-03
41,season_start_x,1.798451e-03
40,season_end_x,1.798451e-03
27,steal_percentage,5.730582e-03
28,block_percentage,7.789122e-03
36,defensive_box_plus_minus,7.947917e-03


In [177]:
def extract_numerical_data(df):
    columns_to_remove = []
    for feature in df.columns.values:
        if type(df.iloc[0][feature])==str:
            columns_to_remove.append(feature)
    return df.drop(columns=columns_to_remove)

def clean_datasets(p_stats, stats_2020, filter_year):
    p_stats = p_stats.dropna()
    stats_2020 = stats_2020.dropna()
    print(p_stats.shape, stats_2020.shape, ' before any filters')
    p_stats = p_stats.loc[p_stats['season_end_x'] >= filter_year]
    stats_2020 = stats_2020.loc[stats_2020['season_end_x'] >= filter_year]
    
    print(p_stats.shape, stats_2020.shape, ' after year filter')
    p_stats = p_stats.loc[p_stats['games_played']>= 35]
    stats_2020 = stats_2020.loc[stats_2020['games_played']>= 35]
    
    print(p_stats.shape, stats_2020.shape, ' after games_played filter')
    p_stats = p_stats.loc[p_stats['cap_usage'] >= 0.01]
    stats_2020 = stats_2020.loc[stats_2020['cap_usage'] >= 0.01]
    print(p_stats.shape, stats_2020.shape, ' after g_league filter')
    
    #add filter on years-played only for p_stats to take out rookies
    p_stats = p_stats.loc[p_stats['years'] > 4]
    print(p_stats.shape, ' p_stats after years_played filter to take out rookies')
    stats_2020_wo_rookies = stats_2020.loc[stats_2020['years'] > 4]
    
    X_train = extract_numerical_data(p_stats)
    X_test = extract_numerical_data(stats_2020)
    X_train = X_train.drop(columns=['Unnamed: 0'])
    X_test = X_test.drop(columns=['Unnamed: 0'])
    X_train = X_train.dropna()
    X_test = X_test.dropna()
    X_train = X_train.loc[X_train['season_end_x'] >= filter_year]
    X_test = X_test.loc[X_test['season_end_x'] >= filter_year]
    y_train = X_train[['cap_usage']]
    y_test = X_test[['cap_usage']]
    X_train = X_train.drop(columns=['cap_usage'])
    X_test = X_test.drop(columns=['cap_usage'])
    X_train = X_train.drop(columns=['salary_cap_x', 'salary_cap_y'])
    print("is the cleaning is successful?:", X_train.columns.values==X_test.columns.values)
    return X_train, X_test, y_train, y_test, p_stats, stats_2020, stats_2020_wo_rookies



In [178]:
X_train, X_test, y_train, y_test, p_stats, stats_2020, wo_rook_2020 = clean_datasets(p_stats, stats_2020, 2015)

(1148, 51) (345, 50)  before any filters
(1148, 51) (345, 50)  after year filter
(1148, 51) (345, 50)  after games_played filter
(1148, 51) (345, 50)  after g_league filter
(989, 51)  p_stats after years_played filter to take out rookies
is the cleaning is successful?: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge

In [179]:
features = ['points', 'made_field_goals', 'minutes_played', 'defensive_rebounds', 'win_shares', 'value_over_replacement_player', 'turnovers']
X_train_new = X_train[features]
X_test_new = X_test[features]

In [17]:
def model_performance(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    print("score for the training set: ", model.score(X_train, y_train))
    print("score for the test set: ", model.score(X_test, y_test))
    print("mean squared error for the training set: ", mean_squared_error(model.predict(X_train), y_train))
    print("mean squared error for the test set: ", mean_squared_error(model.predict(X_test), y_test))

## model building starts here

#### model 1: boosting

In [83]:
import sklearn.ensemble as ske

In [84]:
from sklearn.ensemble import AdaBoostRegressor

In [161]:
model = AdaBoostRegressor(n_estimators=5)
model_performance(model, X_train_new, y_train, X_test_new, y_test)

score for the training set:  0.48130679442679924
score for the test set:  0.2887108532326952
mean squared error for the training set:  0.0035287955736867517
mean squared error for the test set:  0.004728931742609474


C:\Users\risha\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [181]:
train_with_name = pd.concat([p_stats[['name']], X_train_new], axis=1)
test_with_name = pd.concat([stats_2020[['name']], X_test_new], axis=1)

In [163]:
def make_dataframe_of_results(test_df, y_test, model, features):
    #pass in a fitted model
    prediction = pd.DataFrame(columns=['name', 'prediction', 'actual'])
    prediction['name'] = test_df['name']
    prediction['prediction'] = model.predict(test_df[features])
    prediction['actual'] = y_test
    prediction['error'] = prediction['prediction'] - prediction['actual']
    prediction['error_squared'] = prediction['error']*prediction['error']
    return prediction

In [164]:
prediction1 = make_dataframe_of_results(test_with_name, y_test, model, features)
prediction1.loc[prediction1['name']=='Mike Conley']

,name,prediction,actual,error,error_squared
195,Mike Conley,0.113639,0.297889,-0.18425,0.033948


In [165]:
model_linear = LinearRegression()
model_linear.fit(train_with_name[features], y_train)
prediction_linear = make_dataframe_of_results(test_with_name, y_test, model_linear, features)
prediction_linear.loc[prediction_linear['name']=='Mike Conley']

,name,prediction,actual,error,error_squared
195,Mike Conley,0.119661,0.297889,-0.178228,0.031765


#### model 2: random forest regressor

In [166]:
model_rf = ske.RandomForestRegressor(max_depth = 5, min_samples_split=3, min_samples_leaf=1)
model_rf.fit(train_with_name[features], y_train)
prediction_rf = make_dataframe_of_results(test_with_name, y_test, model_rf, features)
model_performance(model_rf, train_with_name[features], y_train, test_with_name[features], y_test)

<ipython-input-166-4ad1f91bae2c>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf.fit(train_with_name[features], y_train)
<ipython-input-17-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.621796731482164
score for the test set:  0.32350848626239215
mean squared error for the training set:  0.0025730084866346977
mean squared error for the test set:  0.004497583306956141


In [53]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

#stadnardiztion
#featuesecltion
#training
#validation
#test
#predict

In [167]:
#attempt1 - bad
pipe = Pipeline([('regressor', ske.RandomForestRegressor())])
samples_split = np.arange(2, 5)
samples_leaf = np.arange(1, 4)

params = [{'regressor__min_samples_split': samples_split, 'regressor__min_samples_leaf':samples_leaf, }]

gridsearch = GridSearchCV(pipe, params, verbose=1).fit(train_with_name[features], np.ravel(y_train))
print('The best score is: ', gridsearch.best_score_)
print('The best parameters are: ', gridsearch.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   52.8s finished


The best score is:  0.4275671041326029
The best parameters are:  {'regressor__min_samples_leaf': 3, 'regressor__min_samples_split': 3}


In [61]:
#attempt2 - better
pipe = Pipeline([('regressor', ske.RandomForestRegressor())])
samples_split = np.arange(2, 5)
samples_leaf = np.arange(1, 4)
                 
params = [{'regressor__min_samples_split': [3], 'regressor__min_samples_leaf': [3], 'regressor__max_features':np.arange(1, 8, 2), 'regressor__max_depth':np.arange(3, 6), 'regressor__n_estimators': np.arange(10, 200, 40)}]

gridsearch = GridSearchCV(pipe, params, verbose=1).fit(train_with_name[features], np.ravel(y_train))
print('The best score is: ', gridsearch.best_score_)
print('The best parameters are: ', gridsearch.best_params_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  4.3min finished


The best score is:  0.44617945494073225
The best parameters are:  {'regressor__max_depth': 5, 'regressor__max_features': 1, 'regressor__min_samples_leaf': 3, 'regressor__min_samples_split': 3, 'regressor__n_estimators': 130}


In [105]:
model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
prediction_rf = make_dataframe_of_results(test_with_name, y_test, model_rf, features)
model_performance(model_rf, train_with_name[features], y_train, test_with_name[features], y_test)

<ipython-input-105-e6916b4a9474>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
<ipython-input-17-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.5204418866317522
score for the test set:  0.4523435032584424
mean squared error for the training set:  0.0029807483065542465
mean squared error for the test set:  0.0037612294338835706


In [106]:
pd.set_option('display.max_rows', None)
prediction_rf.sort_values(by= ['error_squared'])

,name,prediction,actual,error,error_squared
379,Robin Lopez,0.043718,0.043678,0.000041,1.648774e-09
20,D.J. Augustin,0.066352,0.066428,-0.000076,5.825918e-09
214,Markelle Fultz,0.089128,0.089291,-0.000162,2.640341e-08
350,Maxi Kleber,0.073710,0.073300,0.000410,1.678624e-07
220,Langston Galloway,0.066238,0.067192,-0.000954,9.091670e-07
498,Josh Richardson,0.090844,0.092542,-0.001697,2.880678e-06
354,Kevin Knox,0.042256,0.040133,0.002123,4.505537e-06
236,Jerami Grant,0.082884,0.085635,-0.002751,7.567411e-06
386,Thon Maker,0.035747,0.032707,0.003040,9.244298e-06
207,Yogi Ferrell,0.031926,0.028862,0.003064,9.385203e-06


In [224]:
#test on 2020 without rookies
model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
prediction_rf = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_rf, features)
model_performance(model_rf, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

<ipython-input-224-de8b1338c387>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
<ipython-input-17-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.6337141633595518
score for the test set:  0.579279438802607
mean squared error for the training set:  0.0025666786391117997
mean squared error for the test set:  0.003417345727252844


In [191]:
prediction_rf.sort_values(by = ['error_squared'])

,name,prediction,actual,error,error_squared
45,Marco Belinelli,0.053972,0.053566,0.000406,1.650999e-07
278,Evan Fournier,0.156229,0.155763,0.000466,2.167403e-07
696,Tristan Thompson,0.170437,0.169866,0.000572,3.269445e-07
635,Dwight Powell,0.092952,0.094002,-0.001050,1.101822e-06
188,Jordan Clarkson,0.124500,0.123122,0.001378,1.899449e-06
655,Terry Rozier,0.180301,0.182286,-0.001986,3.943294e-06
400,Joe Ingles,0.111880,0.109534,0.002346,5.505131e-06
132,Kentavious Caldwell-Pope,0.076791,0.074118,0.002672,7.141922e-06
11,Kyle Anderson,0.080388,0.083132,-0.002744,7.532052e-06
494,Brook Lopez,0.107987,0.110803,-0.002816,7.931117e-06


In [186]:
wo_rook_2020.loc[wo_rook_2020['name']=='John Collins']

,Unnamed: 0,name,positions,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,box_plus_minus,value_over_replacement_player,Year,player_id_x,salary_x,season,season_end_x,season_start_x,cap_usage,years


## taking out max star players

In [208]:
p_stats_noMAX = p_stats.loc[p_stats['cap_usage'] <= 0.2]
stats_2020_noMAX = stats_2020.loc[stats_2020['cap_usage'] <= 0.2]
X_train_noMAX, X_test_noMAX, y_train_noMAX, y_test_noMAX, p_stats_noMAX_v2, stats_2020_noMAX, wo_rook_2020_noMAX = clean_datasets(p_stats_noMAX, stats_2020_noMAX, 2015)

(817, 51) (306, 50)  before any filters
(817, 51) (306, 50)  after year filter
(817, 51) (306, 50)  after games_played filter
(817, 51) (306, 50)  after g_league filter
(817, 51)  p_stats after years_played filter to take out rookies
is the cleaning is successful?: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


In [209]:
features = ['points', 'made_field_goals', 'minutes_played', 'defensive_rebounds', 'win_shares', 'value_over_replacement_player', 'turnovers']
X_train_noMAX = X_train_noMAX[features]
X_test_noMAX = X_test_noMAX[features]

In [210]:
train_with_name_noMAX = pd.concat([p_stats_noMAX_v2[['name']], X_train_noMAX], axis=1)
test_with_name_noMAX = pd.concat([stats_2020_noMAX[['name']], X_test_noMAX], axis=1)

In [220]:
#test on 2020 without rookies and without max players
model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name_noMAX[features], y_train_noMAX)
prediction_rf = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_rf, features)
model_performance(model_rf, train_with_name_noMAX[features], y_train_noMAX, wo_rook_2020[features], wo_rook_2020['cap_usage'])

<ipython-input-220-c69d36508046>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name_noMAX[features], y_train_noMAX)
<ipython-input-17-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.4757712868997591
score for the test set:  0.36930863285704674
mean squared error for the training set:  0.0014833910140219161
mean squared error for the test set:  0.00512285504323144


In [221]:
prediction_rf.sort_values(by = ['error_squared'])

,name,prediction,actual,error,error_squared
132,Kentavious Caldwell-Pope,0.073772,0.074118,-0.000347,1.203025e-07
45,Marco Belinelli,0.051892,0.053566,-0.001673,2.799642e-06
477,Jeremy Lamb,0.098437,0.096207,0.002230,4.972768e-06
513,Doug McDermott,0.072083,0.067192,0.004891,2.392204e-05
635,Dwight Powell,0.088928,0.094002,-0.005074,2.574127e-05
304,Jerami Grant,0.091843,0.085635,0.006208,3.854259e-05
374,George Hill,0.076410,0.083690,-0.007280,5.299317e-05
375,George Hill,0.076410,0.083690,-0.007280,5.299317e-05
400,Joe Ingles,0.102254,0.109534,-0.007280,5.300436e-05
11,Kyle Anderson,0.075579,0.083132,-0.007553,5.705211e-05


# FINAL MODEL

In [225]:
#test on 2020 data without rookies in the training set or the test set ==> signifcantly better results
model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
prediction_rf = make_dataframe_of_results(wo_rook_2020[features + ['name']], wo_rook_2020['cap_usage'], model_rf, features)
model_performance(model_rf, train_with_name[features], y_train, wo_rook_2020[features], wo_rook_2020['cap_usage'])

<ipython-input-225-5ce155582b10>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf = ske.RandomForestRegressor(max_depth = 5, max_features = 1, min_samples_leaf = 3, min_samples_split = 3, n_estimators = 130).fit(train_with_name[features], y_train)
<ipython-input-17-c491b32f82f9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


score for the training set:  0.6336255892923968
score for the test set:  0.5764656976075486
mean squared error for the training set:  0.0025672993051146986
mean squared error for the test set:  0.003440200627482017
